# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [18]:
%pip install duckdb leafmap lonboard
import duckdb
import leafmap

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [29]:
%pip install duckdb duckdb-engine jupysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00


In [30]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

In [31]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [32]:
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

In [33]:
%%sql

SELECT * FROM duckdb_extensions();

,extension_name,loaded,installed,install_path,description,aliases,extension_version,install_mode,installed_from
0,arrow,False,False,,A zero-copy data integration between Apache Ar...,[],,None,
1,autocomplete,False,False,,Adds support for autocomplete in the shell,[],,None,
2,aws,False,False,,Provides features that depend on the AWS SDK,[],,None,
3,azure,False,False,,Adds a filesystem abstraction for Azure blob s...,[],,None,
4,delta,False,False,,Adds support for Delta Lake,[],,None,
5,excel,False,False,,Adds support for Excel-like format strings,[],,None,
6,fts,False,True,(BUILT-IN),Adds support for Full-Text Search Indexes,[],,STATICALLY_LINKED,
7,httpfs,False,False,,Adds support for reading and writing files ove...,"[http, https, s3]",,None,
8,iceberg,False,False,,Adds support for Apache Iceberg,[],,None,
9,icu,True,True,(BUILT-IN),Adds support for time zones and collations usi...,[],,STATICALLY_LINKED,


In [34]:
%%sql

INSTALL httpfs;
LOAD httpfs;

,Success


In [44]:
import leafmap
import zipfile
import os
import duckdb

# Step 1: Download the nyc_data.zip dataset
zip_url = "https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip"
zip_file = "nyc_data.zip"
output_folder = "nyc_data"

# Download the file using leafmap
leafmap.download_file(zip_url, output=zip_file, overwrite=True)

# Step 2: Extract the ZIP file and verify contents
expected_files = [
    "nyc_census_blocks.csv",
    "nyc_homicides.csv",
    "nyc_neighborhoods.csv",
    "nyc_streets.csv",
    "nyc_subway_stations.csv",
]

if os.path.exists(zip_file):
    try:
        with zipfile.ZipFile(zip_file, "r") as zip_ref:
            zip_ref.extractall(output_folder)
            print(f"Extracted files to {output_folder}")

        # Verify that all expected files are present
        missing_files = [f for f in expected_files if not os.path.exists(os.path.join(output_folder, f))]
        if missing_files:
            print(f"Warning: The following expected files are missing: {missing_files}")
    except zipfile.BadZipFile:
        print("The downloaded file is not a valid ZIP file. Please check the source.")
        exit()
else:
    print(f"ZIP file {zip_file} does not exist.")
    exit()


Downloading...
From: https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip
To: /content/nyc_data.zip
100%|██████████| 8.73M/8.73M [00:00<00:00, 193MB/s]


Extracting files...
Extracted files to nyc_data


In [45]:
url= "https://github.com/opengeos/data/raw/main/duckdb/nyc data. zip
leafmap.download file(url, unzip True)

SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-45-8ea0bb0c5862>, line 1)

In [38]:
%%sql

CREATE TABLE nyc_census_blocks AS SELECT * FROM '/content/nyc_census_blocks.shp'

,Success


In [39]:
%%sql

CREATE TABLE nyc_homicides AS SELECT * FROM '/content/nyc_homicides.shp'

,Success


In [46]:
%%sql

CREATE TABLE nyc_neighborhoods AS SELECT * FROM '/content/nyc_neighborhoods.shp'

,Success


In [47]:
%%sql

CREATE TABLE nyc_streets AS SELECT * FROM '/content/nyc_streets.shp'

,Success


In [49]:
%%sql

CREATE TABLE nyc_subway_stations AS SELECT * FROM '/content/nyc_subway_stations.shp'

,Success


1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [61]:
%%sql

SELECT s.name, s.routes
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_Contains(n.geom, s.geom)
WHERE n.name = 'Little Italy';

,NAME,ROUTES
0,Spring St,6


2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [63]:
%%sql

SELECT DISTINCT n.name, n.boroname
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_Contains(n.geom, s.geom)
WHERE strpos(s.routes,'6') > 0;

,NAME,BORONAME
0,Financial District,Manhattan
1,Little Italy,Manhattan
2,Upper East Side,Manhattan
3,East Harlem,Manhattan
4,Mott Haven,The Bronx
5,Hunts Point,The Bronx
6,South Bronx,The Bronx
7,Chinatown,Manhattan
8,Greenwich Village,Manhattan
9,Gramercy,Manhattan


3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [64]:
%%sql

SELECT Sum(popn_total)
FROM nyc_neighborhoods AS n
JOIN nyc_census_blocks AS c
ON ST_Intersects(n.geom, c.geom)
WHERE n.name = 'Battery Park';

,sum(popn_total)
0,17153.0


4. **What neighborhood has the highest population density (persons/km2)?**


In [65]:
%%sql

SELECT
  n.name,
  Sum(c.popn_total) / (ST_Area(n.geom) / 1000000.0) AS popn_per_sqkm
FROM nyc_census_blocks AS c
JOIN nyc_neighborhoods AS n
ON ST_Intersects(c.geom, n.geom)
GROUP BY n.name, n.geom
ORDER BY popn_per_sqkm DESC LIMIT 2;

,NAME,popn_per_sqkm
0,North Sutton Area,68435.132838
1,East Village,50404.483413


When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [ ]:
%pip install duckdb duckdb-engine jupysql

In [ ]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [ ]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.